# Density Plots

Notebook to explore the density plots of the predicted probability.

Based on this notebook we will implement this logic in the train script to automatically log the density plots for:
* train
* test
* oot 

splits to mlflow

In [ ]:
import os
import pickle
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt
from hydra.utils import instantiate
from hydra import compose, initialize

from my_logger.custom_logger import  logger
from ProcessData.data_splitter import data_splitter
from ProcessData.dataset import Dataset


In [ ]:
initialize(config_path="conf/", version_base=None)
cfg = compose(config_name="config.yaml")

In [ ]:

data_pipeline = instantiate(cfg.data_pipeline)
df = pd.read_csv(cfg.Data_Source)
df = data_pipeline.apply(df.copy())
dataset = Dataset(data=df, data_splitter=data_splitter)

In [ ]:
dataset.X_train[['Datum','next_close_price1', 'close_price1', 'diff']]

In [ ]:
dataset.y_train

In [ ]:
model_name = 'deploy/model.pkl'

current_directory = os.path.dirname(os.path.realpath('scoring_script.py'))
# Construct the relative path to the model file
model_path = os.path.join(current_directory, model_name)
logger.info(f'Get model from {model_path}')
# Check if the model file exists
if not os.path.exists(model_path):
    raise FileNotFoundError(f"Model file '{model_name}' not found in the current directory.")

# Load the model
with open (model_path, 'rb') as m:
    model = pickle.load(m)
print(type(model))

In [ ]:
pred = model.predict_proba(dataset.X_oot)

In [ ]:
sns.kdeplot(pred[:,1], shade=True)
plt.xlabel('Probability')
plt.ylabel('Density')
plt.title('Density Plot of Predicted Probabilities')
plt.show()

In [ ]:


# Assuming you have some predict_proba results stored in a variable called predict_proba_results
# Replace this with your actual predict_proba results
pred_1 = [x if y > 0.5 else 0 for x, y in zip(pred[:, 1], dataset.y_oot)]
pred_1 = [x for x in pred_1 if x != 0]


pred_0 = [x if y < 0.5 else 0 for x, y in zip(pred[:, 1], dataset.y_oot)]
pred_0 = [x for x in pred_0 if x != 0]

# Create a density plot using seaborn
sns.kdeplot(pred_0, shade=True)
sns.kdeplot(pred_1, shade=True)
plt.xlabel('Probability')
plt.ylabel('Density')
plt.title('Density Plot of Predicted Probabilities')
plt.show()
